In [1]:
%pip install "opencompass[full]"
%pip install pytorch transformers datasets "opencompass[full]"

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
  Using cached http://mirrors.aliyun.com/pypi/packages/ee/67/f403d4ae6e9cd74b546ee88cccdb29b8415a9c1b3d80aebeb20c9ea91d96/pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-3qfp__9v/pytorch_10b007cadca54706bc8e277ce87b8cb4/setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a p

安装opencompass：Kaggle上已经为我们准备好了其他常用包，只需安装opencompass用于评测即可。如果不在Kaggle上运行，则还需要安装其他必要包。

# 指令微调

In [1]:
"""
The main program for finetuning LLMs with Huggingface Transformers Library.

ALL SECTIONS WHERE CODE POSSIBLY NEEDS TO BE FILLED IN ARE MARKED AS TODO.
"""

import argparse
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import sys
import torch
from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM
import datasets
from trl import SFTTrainer

In [2]:
# Define the arguments required for the main program.
# NOTE: You can customize any arguments you need to pass in.
@dataclass
class ModelArguments:
    """Arguments for model
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the LLM to fine-tune or its name on the Hugging Face Hub."
        }
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype."
            ),
            "choices": ["bfloat16", "float16", "float32"],
        },
    )
    # TODO: add your model arguments here
    pass


@dataclass
class DataArguments:
    """Arguments for data
    """
    dataset_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the fine-tuning dataset or its name on the Hugging Face Hub."
        }
    )
    # TODO: add your data arguments here

# No Mask Label

In [3]:
# The main function
# NOTE You can customize some logs to monitor your program.
def finetune():
    # TODO Step 1: Define an arguments parser and parse the arguments
    # NOTE Three parts: model arguments, data arguments, and training arguments
    # HINT: Refer to 
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/internal/trainer_utils#transformers.HfArgumentParser
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments
    parser = HfArgumentParser((ModelArguments, DataArguments,TrainingArguments))
    model_args, data_args, training_args = parser.parse_args_into_dataclasses(args=sys.argv)

    # TODO Step 2: Load tokenizer and model
    # HINT 1: Refer to
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/tokenizer#tokenizer
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/qwen2
    # HINT 2: To save training GPU memory, you need to set the model's parameter precision to half-precision (float16 or bfloat16).
    #         You may also check other strategies to save the memory!
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/llama2#usage-tips
    #   * https://huggingface.co/docs/transformers/perf_train_gpu_one
    #   * https://www.53ai.com/news/qianyanjishu/2024052494875.html
    device = 'cuda'
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
    model = AutoModelForCausalLM.from_pretrained(model_args.model_name_or_path,device_map = 'auto',torch_dtype = 'bfloat16')

    # TODO Step 3: Load dataset
    # HINT: https://huggingface.co/docs/datasets/v3.1.0/en/package_reference/main_classes#datasets.Dataset
    dataset = datasets.load_dataset(data_args.dataset_path)
    raw_train_dataset = dataset['train']
    
    # TODO Step 4: Define the data collator function
    # NOTE During training, for each model parameter update, we fetch a batch of data, perform a forward and backward pass,
    # and then update the model parameters. The role of the data collator is to process the data (e.g., padding the data within
    # a batch to the same length) and format the batch into the input required by the model.
    #
    # In this assignment, the purpose of the custom data_collator is to process each batch of data from the dataset loaded in
    # Step 3 into the format required by the model. This includes tasks such as tokenizing the data, converting each token into 
    # an ID sequence, applying padding, and preparing labels.
    # 
    # HINT:
    #   * Before implementation, you should:
    #      1. Clearly understand the format of each sample in the dataset loaded in Step 3.
    #      2. Understand the input format required by the model (https://huggingface.co/docs/transformers/model_doc/qwen2#transformers.Qwen2ForCausalLM).
    #         Reading its source code also helps!

    def data_collator(batch: List[Dict]):
        """
        batch: list of dict, each dict of the list is a sample in the dataset.
        """
        tokenized_input = tokenizer(
            [f"You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer. Instruction:{item['instruction']}.\nInput:{'' if item['input']==None else item['input']}\nOutput: {item['output']}" 
             for item in batch],
            padding='max_length',
            max_length=1024,
            padding_side='left',
            truncation=True,
            add_special_tokens=True,
            return_tensors='pt'
        ).to(device)
        
        # tokenized_output = tokenizer(
        #     [f"Output: {item['output']}" 
        #      for item in batch],
        #     padding='max_length',
        #     max_length=1024,
        #     padding_side='left',
        #     truncation=True,
        #     add_special_tokens=True,
        #     return_tensors='pt'
        # ).to(device)
        
        
        input_ids,attention_mask = tokenized_input['input_ids'],tokenized_input['attention_mask']
        labels = tokenized_input['input_ids']
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

    # TODO Step 5: Define the Trainer
    # HINT: https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        data_collator=data_collator,
        train_dataset=raw_train_dataset
    )

    # Step 6: Train!
    trainer.train()

# Mask Label

In [35]:
# The main function
# NOTE You can customize some logs to monitor your program.
def finetune():
    # TODO Step 1: Define an arguments parser and parse the arguments
    # NOTE Three parts: model arguments, data arguments, and training arguments
    # HINT: Refer to 
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/internal/trainer_utils#transformers.HfArgumentParser
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments
    parser = HfArgumentParser((ModelArguments, DataArguments,TrainingArguments))
    model_args, data_args, training_args = parser.parse_args_into_dataclasses(args=sys.argv)

    # TODO Step 2: Load tokenizer and model
    # HINT 1: Refer to
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/tokenizer#tokenizer
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/qwen2
    # HINT 2: To save training GPU memory, you need to set the model's parameter precision to half-precision (float16 or bfloat16).
    #         You may also check other strategies to save the memory!
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/llama2#usage-tips
    #   * https://huggingface.co/docs/transformers/perf_train_gpu_one
    #   * https://www.53ai.com/news/qianyanjishu/2024052494875.html
    device = 'cuda'
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
    model = AutoModelForCausalLM.from_pretrained(model_args.model_name_or_path,device_map = 'auto',torch_dtype = 'bfloat16')

    # TODO Step 3: Load dataset
    # HINT: https://huggingface.co/docs/datasets/v3.1.0/en/package_reference/main_classes#datasets.Dataset
    dataset = datasets.load_dataset(data_args.dataset_path)
    raw_train_dataset = dataset['train']
    
    # TODO Step 4: Define the data collator function
    # NOTE During training, for each model parameter update, we fetch a batch of data, perform a forward and backward pass,
    # and then update the model parameters. The role of the data collator is to process the data (e.g., padding the data within
    # a batch to the same length) and format the batch into the input required by the model.
    #
    # In this assignment, the purpose of the custom data_collator is to process each batch of data from the dataset loaded in
    # Step 3 into the format required by the model. This includes tasks such as tokenizing the data, converting each token into 
    # an ID sequence, applying padding, and preparing labels.
    # 
    # HINT:
    #   * Before implementation, you should:
    #      1. Clearly understand the format of each sample in the dataset loaded in Step 3.
    #      2. Understand the input format required by the model (https://huggingface.co/docs/transformers/model_doc/qwen2#transformers.Qwen2ForCausalLM).
    #         Reading its source code also helps!

    def data_collator(batch: List[Dict]):
        """
        batch: list of dict, each dict of the list is a sample in the dataset.
        """
        tokenized_input = tokenizer(
            [f"You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer. Instruction:{item['instruction']}.\nInput:{'' if item['input']==None else item['input']}\nOutput: {item['output']}" 
             for item in batch],
            padding='max_length',
            max_length=1024,
            padding_side='left',
            truncation=True,
            add_special_tokens=True,
            return_tensors='pt'
        ).to(device)
        
        tokenized_output = tokenizer.batch_encode_plus(
            [f"Output: {item['output']}" 
             for item in batch],
            padding='max_length',
            max_length=1024,
            padding_side='left',
            truncation=True,
            return_tensors='pt'
        ).to(device)
        
        equal_indices = (tokenized_output['input_ids'] != tokenized_input['input_ids'])
        labels = tokenized_input['input_ids'].clone()
        labels[equal_indices] = -100
        
        
        input_ids,attention_mask = tokenized_input['input_ids'],tokenized_input['attention_mask']
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

    # TODO Step 5: Define the Trainer
    # HINT: https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        data_collator=data_collator,
        train_dataset=raw_train_dataset
    )

    # Step 6: Train!
    trainer.train()

In [4]:
# Pass your training arguments.
# NOTE [IMPORTANT!!!] DO NOT FORGET TO PASS PROPER ARGUMENTS TO SAVE YOUR CHECKPOINTS!!!
base_dir = './model/input/qwen2.5/transformers/0.5b/1'
sys.argv = [
    "--model_name_or_path", base_dir,
    "--torch_dtype", "bfloat16",
    "--output_dir","./output",
    "--dataset_path","./Dataset/alpaca-language-instruction-training",
    "--remove_unused_columns", "False",
    "--per_device_train_batch_size", "4",
    "--do_train", "True",
    "--dataloader_pin_memory","False",
    "--num_train_epochs", "3",
    "--logging_strategy", "steps",
    "--logging_steps","500",
    "--logging_dir","./output/log",
    "--report_to","tensorboard"
]
finetune()

/tmp/ipykernel_3065/2931503463.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.284400
1000,0.250200
1500,0.254100
2000,0.245700
2500,0.252800
3000,0.242200
3500,0.246300
4000,0.230800
4500,0.242300
5000,0.232800


# 评测模型

如果你有多个GPU，可以修改下面的--hf-num-gpus参数来加速评测.

In [2]:
!opencompass \
    --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path model/input/qwen2.5/transformers/0.5b/1 \
    --tokenizer-kwargs padding_side="left" truncation="True" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 1 \
    --work-dir "./base_eval" \
    --debug

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/__init__.py:19: UserWarning: Starting from v0.4.0, all AMOTIC configuration files currently located in `./configs/datasets`, `./configs/models`, and `./configs/summarizers` will be migrated to the `opencompass/configs/` package. Please update your configuration file paths accordingly.
  _warn_about_config_migration()
12/23 18:04:27 - OpenCompass - INFO - Loading mmlu_ppl: /root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/configs/./datasets/mmlu/mmlu_ppl.py
12/23 18:04:27 - OpenCompass - INFO - Loading hellaswag_clean_ppl: /root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/configs/./datasets/hellaswag/hellaswag_clean_ppl.py
12/23 18:04:27 - OpenCompass - INFO - Loading winogrande_ll: /root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/configs/./datasets/winogrande/winogrande_ll.py
12/23 18:04:27 - OpenCompass - INFO - Loading ARC_e_ppl: /root/miniconda3/envs/nlp/lib/python3

In [1]:
!opencompass \
    --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path output/checkpoint-38820 \
    --tokenizer-kwargs padding_side="left" truncation="True" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 1 \
    --work-dir "./evals_masked_sft" \
    --debug

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/__init__.py:19: UserWarning: Starting from v0.4.0, all AMOTIC configuration files currently located in `./configs/datasets`, `./configs/models`, and `./configs/summarizers` will be migrated to the `opencompass/configs/` package. Please update your configuration file paths accordingly.
  _warn_about_config_migration()
12/24 17:11:02 - OpenCompass - INFO - Loading mmlu_ppl: /root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/configs/./datasets/mmlu/mmlu_ppl.py
12/24 17:11:02 - OpenCompass - INFO - Loading hellaswag_clean_ppl: /root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/configs/./datasets/hellaswag/hellaswag_clean_ppl.py
12/24 17:11:02 - OpenCompass - INFO - Loading winogrande_ll: /root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/configs/./datasets/winogrande/winogrande_ll.py
12/24 17:11:02 - OpenCompass - INFO - Loading ARC_e_ppl: /root/miniconda3/envs/nlp/lib/python3